In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1
CUDA version: 11.8
cuDNN version: 8700
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.33.2
Datasets version: 2.14.5


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Wed Jan  3 13:46:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   34C    P8              18W / 250W |   3949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Initial GPU Usage
| ID | GPU  | MEM |
-------------------
|  0 |   0% | 35% |
|  1 | 100% | 63% |
|  2 |  99% | 63% |
|  3 | 100% | 63% |
|  4 | 100% | 63% |
|  5 |   0% | 36% |
|  6 |   0% | 11% |
|  7 |   0% |  0% |
GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 |   0% | 35% |
|  1 | 100% | 63% |
|  2 |  99% | 63% |
|  3 | 100% | 63% |
|  4 |  99% | 63% |
|  5 |   0% | 36% |
|  6 |   0% | 12% |
|  7 |   0% |  0% |


In [8]:
data = pd.read_csv("clean_test_or_not_test.csv" , index_col = 0)
data

,text_clean,label
0,add tests for systemdrawingcommonthis issue tr...,0
1,acctests for packetfabricportresource please k...,0
2,text analytics abstractivesummarizebatchconven...,0
3,see entity recordssee the story in the release...,0
4,editor crash when click mesh instance 3d with ...,0
...,...,...
72563,cdn is downin 987dabdhttpsgithubcomceccunstat...,1
72564,create user registration system using email an...,1
72565,group secure join shows success only after res...,1
72566,plugins not being sourced when using a differe...,1


In [9]:
data_test_set = pd.read_csv("testset_test_or_not_test_clean.csv" , index_col = 0)
data_test_set

,text_clean,label
0,badrecordmac in fips testbuild scan httpsgradl...,0
1,validate shutdown message processing even thou...,0
2,create new flutter project failed on android s...,1
3,upgrade from chromium 103 to chromium 104upgra...,0
4,roachtest followerreadssurvivalzonelocalityreg...,0
...,...,...
8058,autodesk v2 describe the bug autodesk publishe...,1
8059,what happened to sqlite we offer sqlite suppor...,0
8060,setup unittest testing frameworkwe need to uti...,0
8061,use clash for windows cannot be connected in l...,1


In [10]:
# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [11]:
# Drops rows with missing values
data.dropna(inplace=True)

In [12]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,index,text_clean,label
0,0,add tests for systemdrawingcommonthis issue tr...,0
1,1,acctests for packetfabricportresource please k...,0
2,2,text analytics abstractivesummarizebatchconven...,0
3,3,see entity recordssee the story in the release...,0
4,4,editor crash when click mesh instance 3d with ...,0
...,...,...,...
72563,72563,cdn is downin 987dabdhttpsgithubcomceccunstat...,1
72564,72564,create user registration system using email an...,1
72565,72565,group secure join shows success only after res...,1
72566,72566,plugins not being sourced when using a differe...,1


In [13]:
#Drops the index col, better for managing the data.
data.drop(columns= ["index"], inplace = True)

In [14]:
data

,text_clean,label
0,add tests for systemdrawingcommonthis issue tr...,0
1,acctests for packetfabricportresource please k...,0
2,text analytics abstractivesummarizebatchconven...,0
3,see entity recordssee the story in the release...,0
4,editor crash when click mesh instance 3d with ...,0
...,...,...
72563,cdn is downin 987dabdhttpsgithubcomceccunstat...,1
72564,create user registration system using email an...,1
72565,group secure join shows success only after res...,1
72566,plugins not being sourced when using a differe...,1


In [15]:
train , validate , test = train_validate_test_split(data)

In [16]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [17]:
test

,text_clean
label,
1,check the bpmn pipelines to see if formsubmiss...
1,add eslint and enforce code style is your feat...
1,rosskopfyxyz is downin 8f48397httpsgithubcomg...
0,time taken by peer link going down when sonic ...
0,test upgrade 42x to 45 time improvement target...
...,...
1,create a release for rke 142my team is waiting...
0,battle error using moves on some togedemarub75...
1,node build fails with latest version bugdescri...


In [18]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)

separate_test_set = Dataset.from_pandas(data_test_set)
ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds
#ds["separate_test_set"] = separate_test_set

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 7258
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 58054
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 7256
    })
})

In [19]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]
#separate_test_set_dataset = ds["separate_test_set"]

In [20]:
ds["train"][0]

{'text_clean': 'decision tree regressor giving different result for scaled and unscaled datathe decision tree regressor outputs different tree for scaled and unscaled data this is true for the random forest regressor also as per my understanding scaling data should not change the results for decision tree or random forest regressor i am attaching the ipynb file of my analysis in the analysis i have generated a synthetic dataset where the input data is in different scales the decision tree is then made using the unscaled data minmax scaled data and arbitrarily scaled data the arbitrary scaling simply multiplies each input with a suitable value to bring all the inputs in the same order of magnitude it is seen that the tree formed by using the the minmax scaled and arbitrarily scaled data is same but the one formed by the unscaled data is different the same results are reflected when using random forest i am unable to understand this result therefore an explanation would be of help the tr

In [21]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text_clean"], truncation=True)

# Tokenize all the dataset
tokenized_datasets = ds.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/7258 [00:00<?, ? examples/s]

Map:   0%|          | 0/58054 [00:00<?, ? examples/s]

Map:   0%|          | 0/7256 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7258
    })
    train: Dataset({
        features: ['text_clean', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 58054
    })
    validate: Dataset({
        features: ['text_clean', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7256
    })
})

In [23]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [24]:
# Remove unnecessary columns that the model does not expect.
tokenized_datasets = tokenized_datasets.remove_columns(["text_clean"])
# Rename the column to labels because the model expect that.
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# Returns PyTorch tensors instead of lists.
tokenized_datasets.set_format("torch")
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [25]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validate"], batch_size=8, collate_fn=data_collator
)

In [26]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 446]),
 'token_type_ids': torch.Size([8, 446]),
 'attention_mask': torch.Size([8, 446])}

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from torch.optim import AdamW

# The same optimizer as used by "Trainer"
optimizer = AdamW(model.parameters(), lr=5e-5)

In [29]:
from transformers import get_scheduler

# Learning rate scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

21771


In [30]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [31]:
from tqdm.auto import tqdm

# To see when training is finished.
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/21771 [00:00<?, ?it/s]